**Bin Claculator**

- Purpose:  calculating variable-width bins defined by fixed # of ON events for your light curve.

- Input: output file contains **event time, energy, and theta square** produced by *modified flute*.

- Output: bin-edge arrays for *flute*, modify your flute.rc

# Loading input files

## import modules

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

numpy is a useful module to handle arrays.

matplotlib.pyplot is a module to visualize your calculation.

## Reading input

In [ ]:
input_file1 = np.loadtxt('',delimiter=',',skiprows=1)

`input_file1` contains data as follows

$$
\left[\begin{array}{ccc}
            % 横並びは&を挟む
            [T_0 & E_0 & \theta^2_0] \\
           [T_1 & E_1 & \theta^2_1] \\
            & ... & &\\
            % 水平線は\\hlineを使う
             [T_{N-1} & E_{N-1} & \theta^2_{N-1}] \\
        \end{array}\right] \quad
$$

Where, T is an event time (MJD), E is an energy (GeV), $\theta^2$ is a theta square (deg$^2$), and N is the number of ON events

In [ ]:
print input_file1 

One can access any data by `input_file[N,M]`. 

where M is an index for Time, Energy, and Theta square and N is an index for events

e.g.: If you want to access all the event times, you can type as belows

In [ ]:
time_1 = input_file1[:,0]
energy_1 = input_file1[:,1]

## Hands-on 1: Load data by yourself!

Please try loading data for different wobble offset angle.

You can comment out bellows and modify it.

In [ ]:
#input_file2 = np.loadtxt('',delimiter=',',skiprows=1)
#time_2  = input_file2[]
#energy_2 = input_file2[]

# Making a plot

Let's visualize your data using *pyplot*

if you want to make graph a from combinations of values, e.g. (X, Y), please use `pyplot.plot` as `pyplot.plot(X,Y)`

but in this notebook, pyplot can be used as `plt`. So use `plt.pyplot(X,Y)` instead!

In [ ]:
plt.plot(time_1,energy_1,'o')
plt.show()

If you want to make it beautiful, see references.

Japanese: https://qiita.com/KntKnk0328/items/5ef40d9e77308dd0d0a4

English: https://matplotlib.org/3.3.4/api/_as_gen/matplotlib.pyplot.plot.html

## Hands-on 2: Make full plot!

Observation data with different wobble offset angle is still missing in the above plot!

You can add points to the plot before `plt.show()`

Please makr a full plot with both stardard and differecnt wobble offset angle observations.

In [ ]:
plt.plot(time_1,energy_1,'o')
#plt.plot()
#plt.xlabel('')
#plt.ylabel('')
plt.show()

# Calculating bins

You loaded your data and confirmed events by a plot.

So the next is **binning calculations!**

Here, we want to calculate bins, in other words, time durations of flux calculation.

In order to keep statistical presition, we want to hold sevetal ON events in each bin.

Simply speaking, a statstical uncertainty of flux delives from the number of ON events.

A relative uncertainty $\sigma_{rel}$ can be estimated by $\sigma_{rel} = \sigma/\mu = \sqrt{N}/N = 1/\sqrt{N}$ from the possion statistics.

This time, let's dicide the number of events in a bin keeping **the relative uncerataiy is less than 50\%**.

## Hands-on 3: How many events do we need?

Simple question: How many events do we need to keep relative uncertainty is less than 50%.

Please decide the number of ON events.

In [ ]:
#n_of_on_evts = 

## Calculating bin-edges using numpy

What we want to here is following.

- Sorting ON events in order of a event time.

- Dividing ON events every N events. N is value you defidend in the above!

- Calculating bin edges from binned events.

First we need combining the events 

In [ ]:
event_time_all = np.hstack([time_1,time_2])

And sort them by event time.

In [ ]:
event_time_all_sorted = np.sort(event_time_all)

Cheak the number of events.

In [ ]:
print 'N of input1',len(time_1)
print 'N of input2',len(time_2)
print 'N of total events',len(event_time_all_sorted)

In case of IC310, ON event detected within a very short duration.

So we need to remove the events which has same event time.

To do so, use `numpy.unique()`

**NOTE:** This temporal way to calculate bin edges. we will use all the events in later step.

Before going to the bin calculation, we need to get number of resultant bins defined by `n_of_on_evts`

In [ ]:
event_time_cleaned = np.unique(event_time_all_sorted)
m_bin = len(event_time_cleaned) / n_of_on_evts
print 'Number of bins, M',m_bin 
surplus = len(event_time_cleaned) % n_of_on_evts
print 'Surplus',surplus

This surpluse of events will be added at the end of calculation. Don't worry!

### Dividing events in every N events.

For simplisity, prepare a time array without surplus events.

In [ ]:
time_without_surplus = event_time_cleaned[:-surplus]

In [ ]:
binned_events = time_without_surplus.reshape((m_bin,n_of_on_evts))

`ndarray.reshape` is a very powerful function.

Now `binned_events` contains arrays as indicated as belows 


$$
\left[\begin{array}{cccc}
            % 横並びは&を挟む
            [T^0_0 & T^0_1 & ... & T^0_{N-1}] \\
            [T^1_0 & T^1_1 & ... & T^1_{N-1}] \\
            & & ... &\\
            % 水平線は\\hlineを使う
             [T^{M-1}_0 & T^{M-1}_1 & ... & T^{M-1}_{N-1}] \\
        \end{array}\right] \quad
$$
Where $T^i_j$ is $j$ th event in the $i$ th bin.

Compare before and after calculation!

In [ ]:
print time_without_surplus

In [ ]:
print binned_events

Finally, binedges $B$, which we want to calculate is the following.

$$
\begin{align}
B_0 &= T_0^0 \\
B_1 &= \frac{T_0^1 + T_{N-1}^{0}}{2} \\
B_2 &=  \frac{T_0^2 + T_{N-1}^{1}}{2}  \\
...\nonumber \\
B_i &=  \frac{T_0^{i} + T_{N-1}^{i-1}}{2}  \\
...\nonumber \\
B_{M-2} &=  \frac{T_0^{M-2} + T_{N-1}^{M-3}}{2}  \\
B_{M-1} &=  \frac{T_0^{M-1} + T_{N-1}^{M-2}}{2}  \\
B_{M} &= T_{N-1}^{M-1} \\
\end{align}
$$

### Hands-on extra 1: Claculate $B_i$ by a *for loop*

### Calculating $B_i$ by a numpy function

In [ ]:
ts_n_1 = binned_events.max(axis=1)

This correspoinds $T_{N-1}^h$ in the above, because we have sorted events!!!

In [ ]:
binned_events[:,n_of_on_evts-1]

The above is the same.

In [ ]:
ts_0 = binned_events.min(axis=1)

This correspoinds $T_{0}^h$ in the above.

So the answer is the following.

In [ ]:
B_0 = ts_0[0] # B_0
B_i = ts_0[1:] / 2. +ts_n_1[:-1] / 2. # B_i
B_M_1 =  ts_n_1[-1] # B_M-1

`array[-1]` can pick up the value at the end of the array.

`array[1:]` will be new array without `array[0]`.

`array[:-1]` will be new array without `array[-1]`.

In [ ]:
print B_0
print B_i
print B_M_1

In [ ]:
time_bins = np.hstack([np.array(B_0), B_i , np.array(B_M_1)])

In [ ]:
time_bins

Don't forget to include the surplus.

In [ ]:
time_bins[-1] = event_time_all_sorted[-1]

Confirm the end of the array!

In [ ]:
time_bins

Now, we are ready to make outputs for *flute*, before doing that let's check by a plot!

In [ ]:
fig=plt.figure(figsize=(15,5))

plt.plot(time_1,energy_1,'o',label='wobble offset =  deg')
plt.plot(time_2,energy_2,'o',label='wobble offset =  deg')
for i in time_bins:
    plt.plot([i,i],[0,25000],'k--',alpha=0.5)

#plt.xlim(56244.10-0.02,56244.10+0.02)
#plt.ylim(0,25000)
#plt.xlabel('')
#plt.ylabel('')
#plt.legend()
#plt.savefig('')
plt.show()

### Hands-on 4: Complete the above plot for the presentation!

# Producing output for *flute*

What we really want to prepare is the arrays of begining time and end time.

So let's reshape arrays as you can copy and paste.

The format is...

`flute.LCbinlowedge: 55478.177256, 55508.191016, 55544.072045, 55567.053534, 55588.952311, 55595.833718`

` flute.LCbinupedge: 55478.219809, 55508.233503, 55544.108013, 55567.078971, 55588.983142, 55595.848872`

### Hands-on 5: Do it yourself

For LCbinlowedge 

In [ ]:
with open('','w') as f:
    f.write('flute.LCbinlowedge: ')
    for i in ts_0:
        f.write('{}, '.format(i))

In [ ]:
#%less output_example_lowedge.txt

For LCbinupedge